In [0]:
# Caso necessário instalar a biblioteca unidecode.

#%pip install unidecode

In [0]:
from pyspark.sql.functions import   to_date, udf, col, isnan, when, count, datediff
from pyspark.sql.types import StringType
import re
from unidecode import unidecode

In [0]:
# Importação dos dados da camada bronze para o dataframe.

df_silver = spark.read.format("delta").load("s3a://grao-direto-mmk/bronze/grain_logistic_shipping")
display(df_silver)

In [0]:
# Primeira correção no dataframe será os nomes das colunas.
# Utilizando a função unidecode para a padronização dos caracteres.
# Após isso, renomear algumas colunas que não ficaram no padrão.

df_silver.columns

In [0]:
# Lista recebe o nome das colunas após o tratamento.

novos_nomes = []

for nome_coluna in df_silver.columns:
    nome_tratado = nome_coluna.lower().replace(" ", "_")
    nome_tratado = unidecode(nome_tratado)
    novos_nomes.append(nome_tratado)

df_silver = df_silver.toDF(*novos_nomes)

In [0]:
# Ajuste final dos nomes.

df_silver = df_silver.withColumnRenamed("dataenvio", "data_envio") \
                     .withColumnRenamed("dataentrega", "data_entrega") \
                     .withColumnRenamed("avaliacaoentrega", "avaliacao_entrega")\
                     .withColumnRenamed("peso_g", "peso_gramas")

In [0]:
# Checagem: Ok

df_silver.columns

In [0]:
# Função para limpar e converter data em texto pt-BR para "dd/MM/yyyy"
# Uso de regex para realizar o tratamento das datas.


def limpar_data(data_str):
    if data_str is None:
        return None
    
    # Dicionário com os nomes do meses e seus repectivos valore numéricos
    meses = {
        'janeiro': '01', 'fevereiro': '02', 'março': '03',
        'abril': '04', 'maio': '05', 'junho': '06',
        'julho': '07', 'agosto': '08', 'setembro': '09',
        'outubro': '10', 'novembro': '11', 'dezembro': '12'
            }

    # Uso do regex para remover os nomes de dias da semana:
        # Remove qualquer dia com final '-feira,'
        # Remove qualquer dia escrito de outra forma:
            # Ex: 'segunda,', 'ter,' , 'sábado,', 'domingo,' ....(sempre terminando com a vírgula)

    data_str = re.sub(r'^\w+-feira,\s*', '', data_str)
    data_str = re.sub(r'^\w+,\s*', '', data_str)

    # Com isso tenho uma lista no formato ['01','agosto','2023']
    partes = data_str.strip().split(' de ')
    if len(partes) != 3:
        return None

    # Direciono os valores para as variáveis
        # dia: número de dois dígitos;
        # meses: uso o dicionário para alocar o valor numérico do respectivo mês;
        # ano: número de quatro dígitos.
    dia = partes[0].zfill(2)
    mes = meses.get(partes[1].lower().strip())
    ano = partes[2].strip()

    if not mes:
        return None
    
    # Retorno uma string no fomato dd/MM/yyyy
    return f"{dia}/{mes}/{ano}"

In [0]:
# Converter a função criada para o UDF:
limpar_data = udf(limpar_data, StringType())

# Aplicar a função/UDF:
df_bronze = df_bronze.withColumn("data_envio", limpar_data("data_envio"))
df_bronze = df_bronze.withColumn("data_entrega", limpar_data("data_entrega"))

# Conversão para o tipo Date:
df_bronze = df_bronze.withColumn("data_envio", to_date("data_envio", "dd/MM/yyyy"))
df_bronze = df_bronze.withColumn("data_entrega", to_date("data_entrega", "dd/MM/yyyy"))

In [0]:
# Ajuste dos tipos de dados das colunas:
# OBS: id_envio será atribuído o tipo str por conta de ser coluna de ID (evitar perder dados).

df_bronze = (df_bronze
            .withColumn("ligacoes_do_cliente", col("ligacoes_do_cliente").cast("int"))
            .withColumn("avaliacao_do_cliente", col("avaliacao_do_cliente").cast("int"))
            .withColumn("preco", col("preco").cast("float"))
            .withColumn("qtd_itens", col("qtd_itens").cast("int"))
            .withColumn("desconto", col("desconto").cast("int"))
            .withColumn("peso_gramas", col("peso_gramas").cast("int"))
            .withColumn("chegou_no_tempo", col("chegou_no_tempo").cast("int"))
            .withColumn("avaliacao_entrega", col("avaliacao_entrega").cast("int"))
            )

In [0]:
# Checagem Schema: OK

df_bronze.printSchema()


In [0]:
# Visualização da tabela:

display(df_bronze)

In [0]:
# Checagem de valores nulos:
#      |--> sem valores nulos

display(df_bronze.summary("count"))

In [0]:
# Checagem de linhas duplicadas no DataFrame:
#   |--> encontrado 4 linhas duplicadas.

total = df_bronze.count()
sem_duplicatas = df_bronze.dropDuplicates().count()

print(f"Total de linhas: {total}")                      # Total de linhas no DF
print(f"Total sem duplicatas: {sem_duplicatas}")        # Total de linhas duplicadas

if total > sem_duplicatas:
    print(f"Tem {total - sem_duplicatas} linhas duplicadas.")
else:
    print("DataFrame limpo de duplicadas.")

In [0]:
# Limpeza de linhas duplicadas: Ok

df_bronze = df_bronze.dropDuplicates()

### Checagem nos dados das colunas.

Objetivo: tentar encontrar alguma anomalia nas informações\
Ex: elementos da coluna ser = A,B,C,D e encontrar um valor igual a Z.

In [0]:
# Checagem coluna corredor_de_armazenagem:
#   |--> Normal

display(df_bronze.groupBy("corredor_de_armazenagem").count().orderBy("corredor_de_armazenagem", ascending=True))

In [0]:
# Checagem coluna metodo_de_envio:
#   |--> Normal

display(df_bronze.groupBy("metodo_de_envio").count())

In [0]:
# Checagem coluna ligacoes_do_cliente:
#   |--> Normal

display(df_bronze.groupBy("ligacoes_do_cliente").count().orderBy("ligacoes_do_cliente", ascending=False))

In [0]:
# Checagem coluna avaliacao_do_cliente:
#   |--> Normal

display(df_bronze.groupBy("avaliacao_do_cliente").count().orderBy("avaliacao_do_cliente", ascending=False))

In [0]:
# Checagem coluna importancia:
#   |--> Normal

display(df_bronze.groupBy("importancia").count().orderBy("count", ascending=False))

In [0]:
display(df_bronze.groupBy("genero").count())

In [0]:
display(df_bronze.groupBy("desconto").count())

In [0]:
display(df_bronze.groupBy("chegou_no_tempo").count())

In [0]:
# temos 404 casos de #REF! (linha 18):
display(df_bronze.groupBy("destino").count().orderBy("count", ascending=False))

In [0]:
df_bronze = df_bronze.withColumn(
    "destino",
    when(col("destino") == "#REF!", "Indefinido").otherwise(col("destino"))
                                )


In [0]:
display(df_bronze.groupBy("destino").count().orderBy("count", ascending=False))

In [0]:
display(df_bronze.groupBy("avaliacao_entrega").count().orderBy("avaliacao_entrega", ascending=False))

In [0]:
# Adicionando as colunas de 'valor_total' (preco * qtd_itens) e 'tempo_entrega_dias' (tempo que levou para chegar a carga);

df_bronze = (df_bronze.withColumn('valor_total',col('preco') * col('qtd_itens'))
                  .withColumn("tempo_entrega_dias", datediff("data_entrega", "data_envio"))
           )

In [0]:
display(df_bronze)

In [0]:
df_bronze.printSchema()

In [0]:
df_bronze.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("s3a://grao-direto-mmk/silver/grain_logistic_shipping_silver")

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS silver")

df_bronze.write.format("delta")\
            .option("mergeSchema", "true") \
            .mode("overwrite") \
            .saveAsTable("silver.grain_logistic_shipping_silver")

In [0]:
%sql
SELECT * FROM silver.grain_logistic_shipping_silver